# Intro to Quantum Computing with Python and Q\# <br> [Azure Community Live](https://www.youtube.com/watch?v=NdROr0zyMrI) #

Let's use Python interoperability for the [Quantum Development Kit](https://github.com/Microsoft/Quantum) to introduce users to Q\# and quantum computing more generally.

---


Installation instructions for running this notebook on your machine can be found [here](https://docs.microsoft.com/en-ca/quantum/install-guide/python?view=qsharp-preview).

Let's get started by loading the Python package for Q# interoperability called `qsharp`.

In [2]:
import qsharp
qsharp.component_versions()

{'iqsharp': LooseVersion ('0.11.2003.3107'),
 'Jupyter Core': LooseVersion ('1.2.36563.0'),
 '.NET Runtime': LooseVersion ('.NETCoreApp,Version=v3.1'),
 'qsharp': LooseVersion ('0.11.2004.2825')}

## What is a qubit?

A single qubit can be represented by a vector of 2 complex numbers* like this:

$\left|{x}\right\rangle = \left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

<small>* some conditions apply</small>

## What can we _do_ with a qubit?

You can do three types of things with qubits:

- Prepare a qubit
- Do operations with a qubit
- Measure a qubit : returns a 0 or 1

### Preparing a qubit

In [3]:
prepare_qubit = qsharp.compile("""
open Microsoft.Quantum.Diagnostics;

operation PrepareQubit() : Unit {
    using (qubit = Qubit()) {     // We want 1 qubit to use for our task
            DumpMachine();         // Print out what the simulator is keeping a record of
    }
}
""")

In [4]:
prepare_qubit.simulate()

|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖

()

You can read the above output like the vector we wrote above, where the first column is the index, the second is the real part of the vector at that position, and the second is the complex part of that vector entry.

What does `DumpMachine` tell us?

```
|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖
```

This is the same ket we saw earlier!

$\left[\begin{matrix} 1 + 0\times i \\0 + 0\times i \end{matrix}\right]$

## Case Study: Generating quantum random numbers

```Q#
// demo.qs
namespace AzureCommLive.Demo {
    operation Qrng() : Result {
        using (qubit = Qubit()) {   // Preparing 
            H(qubit);               // Operation 
            return MResetZ(qubit);  // Measure and reset 
        }
    }
}
```

We can see that this is a '.qs' file, so it is a Q# source file and it has a number of operations and functions we can import:

In [5]:
from AzureCommLive.Demo import Qrng

## Understanding `Qrng`

We can use built-in documentation as a resource.

In [6]:
?Qrng

That tells us what we can **do** with `Qrng`:

In [7]:
[Qrng.simulate(verbose=False) for _ in range(10)]

[1, 1, 1, 0, 1, 1, 1, 0, 0, 1]

### Randomness brought to you by superposition!

We can use `DumpMachine` again to understand see what the `H` operation does to our qubit.

In [8]:
Qrng.simulate(verbose=True)

Here is what the simulator uses to record a qubit in the 0 state:
|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖 
After using H(qubit) to create a superposition state:
|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

0

### So that's nice, how about more qubits?!

## Operations with multiple qubits can create entanglement

Using Q# with Python, we can also explore other quantum development concepts, like **entanglement**.

In [9]:
from AzureCommLive.Demo import EntangleQubits
results = EntangleQubits.simulate(verbose=True)

State of inital two qubits:
|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0 + 0𝑖 
After entangling the two qubits:
|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

What does `DumpRegister` tell us this time?
```
|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖
```
No matter how many times we run, both measurements are equal to each other.

In [10]:
[EntangleQubits.simulate(verbose=False) for _ in range(10)]

[(0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1)]


## Toy quantum algorithm: Deutsch–Jozsa 

_If I had a function that had one bit input and output, how many different options would I have?_

<figure style="text-align: center;">
    <img src="media/twobit.png" width="50%">
    <caption>
      <br>  
        <strong>Diagram of all possible one bit functions</strong>
    </caption>
</figure>

>#### Deutsch–Jozsa Algorithim ####
>**Problem statement:**
>
>* **GIVEN:** A black box quantum operation which takes 1 input bit and produces either a 0 or a 1 as output. We are promised that the box is either _constant_ or _balanced_. 
>					
>* **GOAL:** to determine if the box output is _constant_ or _balanced_ by evaluating sample inputs.

Deutsch–Jozsa can do this in **one** query to the black box!

<figure style="text-align: center;">
    <img src="media/twobitDJ.png" width="40%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

## Let's load some Q\# code from the directory...

In [11]:
is_zero_oracle_balanced = qsharp.compile("""
open AzureCommLive.DeutschJozsa;

operation IsZeroOracleBalanced(): Bool {
    return IsOracleBalanced(ZeroOracle);
}
""")

In [12]:
is_zero_oracle_balanced.simulate()

False

<figure style="text-align: center;">
    <img src="media/twobitDJ.png" width="40%">
    <caption>
      <br>  
        <strong>Global property of the one bit functions: Constant or Balanced</strong>
    </caption>
</figure>

In [13]:
is_not_oracle_balanced = qsharp.compile("""
open AzureCommLive.DeutschJozsa;

operation IsNotOracleBalanced(): Bool {
    return IsOracleBalanced(NotOracle);
}
""")

In [14]:
is_not_oracle_balanced.simulate()

True

## Putting it all together: One query, one answer!

In [15]:
from AzureCommLive.DeutschJozsa import RunDeutschJozsaAlgorithm

In [16]:
RunDeutschJozsaAlgorithm.simulate(verbose=False)

All tests passed!


()

In [17]:
RunDeutschJozsaAlgorithm.simulate(verbose=True)

The ZeroOracle is Balanced: False
The OneOracle is Balanced: False
The IdOracle is Balanced: True
The NotOracle is Balanced: True
All tests passed!


()

---

## Helpful diagnostics :)

In [18]:
for component, version in sorted(qsharp.component_versions().items(), key=lambda x: x[0]):
    print(f"{component:20}{version}")

.NET Runtime        .NETCoreApp,Version=v3.1
Jupyter Core        1.2.36563.0
iqsharp             0.11.2003.3107
qsharp              0.11.2004.2825


In [19]:
import sys
print(sys.version)

3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:34:03) [MSC v.1916 64 bit (AMD64)]
